In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
max_steps = 1001
image_num = 3000
DIR = "$HOME/Study/project_ai/tensorflow"
METADATA_DIR = DIR + "/projector"
METADATA_FILE = METADATA_DIR + "/metadata.tsv"

sess = tf.Session()
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name='embedding')
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)
        
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y = tf.placeholder(tf.float32, [None, 10], name="y-input")
    
with tf.name_scope("input_reshape"):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image("input", image_shaped_input, 10)
    
with tf.name_scope("layer"):
    with tf.name_scope("weights"):
        W = tf.Variable(tf.zeros([784, 10]), name="W")
        variable_summaries(W)
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros([10]), name="b")
        variable_summaries(b)
    with tf.name_scope("wx_input_b"):
        wx_plus_b = tf.matmul(x, W) + b
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)
        
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y, logits=predictiondiction))
    tf.summary.scalar("loss", loss)
with tf.name_scope("train"):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
sess.run(tf.global_variables_initializer())

with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)
        
if tf.gfile.Exists(METADATA_FILE):
    tf.gfile.DeleteRecursively(METADATA_FILE)
    
with open(METADATA_FILE, 'w') as f:
    labels = sess.run(tf.argmax(mnist, test.labels[:], 1))
    for i in range(image_num):
        f.write(str(labels[i]) + "\n")
        
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(METADATA_DIR, sess.graph)
saver = tf.train.Saver()

config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = METADATA_FILE
embed.sprite.image_path = METADATA_DIR + "/mnist_10k_sprite.png"
embed.sprite.single_image_dim.extend([28, 28])
projector.visualize_embeddings(projector_writer.config)

for i in range(max_steps):
    batch_xs, batch = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary, _ = sess.run([merged ,train_step], feed_dict={
        x: batch_xs, y:batch_ys}, options=run_options, run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
    projector_writer.add_summary(summary, i)
    
    if i%100 == 0:
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Iter " + str(i) + ", Testing Accuracy= " + str(acc))
        
saver.save(sess, METADATA_DIR + "/a_model.ckpt", global_step=max_steps)
projector_writer.close()
sess.close()    